In [1]:
from IPython.display import Image, display

from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import MessagesState
from langgraph.graph import START, END, StateGraph
from langgraph.prebuilt import tools_condition, ToolNode
import time 
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

class CustomState(MessagesState):
    forked: bool

# Node
def node1(state: CustomState):
    # print(state.keys())
    # print("Node 1: This is the first node in the graph.")
    time.sleep(10)  # Simulate some processing time
    return {"messages": AIMessage(content="Hello from Node 1! Forked: " + str(state["forked"]))}
def node2(state: CustomState):
    # print("Node 2: This is the second node in the graph.")
    time.sleep(10)  # Simulate some processing time
    return {"messages": AIMessage(content="Hello from Node 2! Forked: " + str(state["forked"]))}
def node3(state: CustomState):
    # print("Node 3: This is the third node in the graph.")
    time.sleep(10)  # Simulate some processing time
    return {"messages": AIMessage(content="Hello from Node 3! Forked: " + str(state["forked"]))}
def node4(state: CustomState):
    # print("Node 4: This is the fourth node in the graph.")
    time.sleep(10)  # Simulate some processing time
    return {"messages": AIMessage(content="Hello from Node 4! Forked: " + str(state["forked"]))}

# Graph
builder = StateGraph(CustomState)

# Define nodes: these do the work
builder.add_node("node1", node1)
builder.add_node("node2", node2)
builder.add_node("node3", node3)
builder.add_node("node4", node4)
# Define edges: these determine the control flow
builder.add_edge(START, "node1")
builder.add_edge("node1", "node2")
builder.add_edge("node2", "node3")
builder.add_edge("node3", "node4")
builder.add_edge("node4", END)


memory = MemorySaver()
graph = builder.compile(checkpointer=MemorySaver())

# Show
# display(Image(graph.get_graph(xray=True).draw_mermaid_png()))

In [2]:
initial_input = {"messages": HumanMessage(content="Human input to start the graph."),
                  "forked": False}

thread = {"configurable": {"thread_id": "11"}}

# Run the graph until the first interruption
for event in graph.stream(initial_input, thread, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Human input to start the graph.
================================== Ai Message ==================================

Hello from Node 1! Forked: False
================================== Ai Message ==================================

Hello from Node 2! Forked: False
================================== Ai Message ==================================

Hello from Node 3! Forked: False
================================== Ai Message ==================================

Hello from Node 4! Forked: False


In [3]:
graph.get_state({'configurable': {'thread_id': '11'}})

StateSnapshot(values={'messages': [HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='c9fb3512-1c38-4c3e-bbee-ea0efb110e5e'), AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='be40b70b-c389-4dde-aa01-96159228cf0e'), AIMessage(content='Hello from Node 2! Forked: False', additional_kwargs={}, response_metadata={}, id='687a6f86-a3db-44f1-a62b-360ba8ccee93'), AIMessage(content='Hello from Node 3! Forked: False', additional_kwargs={}, response_metadata={}, id='a66e9319-c57d-4190-9400-a61852a77f88'), AIMessage(content='Hello from Node 4! Forked: False', additional_kwargs={}, response_metadata={}, id='1cce4197-9df0-474a-aa86-2b91e2e2ac50')], 'forked': False}, next=(), config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f03c392-a110-6f6e-8004-69d0e2ad2f5e'}}, metadata={'source': 'loop', 'writes': {'node4': {'messages': AIMessage(content='Hello from Node 4! Fo

In [4]:
all_states = [s for s in graph.get_state_history(thread)]
len(all_states)
# print("All states in the graph:")
# for state in all_states:
#     print(state)

6

## Replaying (Obsolete - now is Resuming) 
[Link](https://github.com/langchain-ai/langgraph/discussions/3015)

In [5]:
to_replay = all_states[2]
to_replay

StateSnapshot(values={'messages': [HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='c9fb3512-1c38-4c3e-bbee-ea0efb110e5e'), AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='be40b70b-c389-4dde-aa01-96159228cf0e'), AIMessage(content='Hello from Node 2! Forked: False', additional_kwargs={}, response_metadata={}, id='687a6f86-a3db-44f1-a62b-360ba8ccee93')], 'forked': False}, next=('node3',), config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f03c391-e23d-6b53-8002-96c363afed58'}}, metadata={'source': 'loop', 'writes': {'node2': {'messages': AIMessage(content='Hello from Node 2! Forked: False', additional_kwargs={}, response_metadata={}, id='687a6f86-a3db-44f1-a62b-360ba8ccee93')}}, 'step': 2, 'parents': {}, 'thread_id': '11'}, created_at='2025-05-29T03:00:46.495419+00:00', parent_config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkp

In [6]:
to_replay.values

{'messages': [HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='c9fb3512-1c38-4c3e-bbee-ea0efb110e5e'),
  AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='be40b70b-c389-4dde-aa01-96159228cf0e'),
  AIMessage(content='Hello from Node 2! Forked: False', additional_kwargs={}, response_metadata={}, id='687a6f86-a3db-44f1-a62b-360ba8ccee93')],
 'forked': False}

In [7]:
to_replay.next

('node3',)

In [8]:
to_replay.config

{'configurable': {'thread_id': '11',
  'checkpoint_ns': '',
  'checkpoint_id': '1f03c391-e23d-6b53-8002-96c363afed58'}}

In [9]:
for event in graph.stream(None, to_replay.config, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================== Ai Message ==================================

Hello from Node 2! Forked: False
================================== Ai Message ==================================

Hello from Node 3! Forked: False
================================== Ai Message ==================================

Hello from Node 4! Forked: False


In [10]:
all_states = [s for s in graph.get_state_history(thread)]
print(len(all_states))
# print("All states in the graph:")
# for state in all_states:
#     print(state)

8


## Forking

In [11]:
to_fork = all_states[4]
to_fork.values["messages"]

[HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='e4a67f5c-fb68-4653-ab6b-021d66a342bc'),
 AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='b9b8638a-1f31-429b-94e6-89fce9f56ae5'),
 AIMessage(content='Hello from Node 2! Forked: False', additional_kwargs={}, response_metadata={}, id='722e38a9-d19d-48b7-81cc-2d009d4f8d6d')]

In [12]:
to_fork.config

{'configurable': {'thread_id': '11',
  'checkpoint_ns': '',
  'checkpoint_id': '1f03c345-12c9-631d-8002-04d7f52d01f1'}}

In [13]:
fork_config = graph.update_state(
    to_fork.config,
    {
        "messages": [HumanMessage(content='Human forked', 
                               id=to_fork.values["messages"][-1].id)],
        "forked": True
    },
)

In [14]:
fork_config

{'configurable': {'thread_id': '11',
  'checkpoint_ns': '',
  'checkpoint_id': '1f03c348-bb06-6281-8003-c4786df22e61'}}

In [15]:
all_states = [state for state in graph.get_state_history(thread) ]
all_states[0].values["messages"]

[HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='e4a67f5c-fb68-4653-ab6b-021d66a342bc'),
 AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='b9b8638a-1f31-429b-94e6-89fce9f56ae5'),
 HumanMessage(content='Human forked', additional_kwargs={}, response_metadata={}, id='722e38a9-d19d-48b7-81cc-2d009d4f8d6d')]

In [16]:
len(all_states)

9

In [17]:
graph.get_state({'configurable': {'thread_id': '11'}})

StateSnapshot(values={'messages': [HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='e4a67f5c-fb68-4653-ab6b-021d66a342bc'), AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='b9b8638a-1f31-429b-94e6-89fce9f56ae5'), HumanMessage(content='Human forked', additional_kwargs={}, response_metadata={}, id='722e38a9-d19d-48b7-81cc-2d009d4f8d6d')], 'forked': True}, next=('node3',), config={'configurable': {'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f03c348-bb06-6281-8003-c4786df22e61'}}, metadata={'source': 'update', 'writes': {'node2': {'messages': [HumanMessage(content='Human forked', additional_kwargs={}, response_metadata={}, id='722e38a9-d19d-48b7-81cc-2d009d4f8d6d')], 'forked': True}}, 'step': 3, 'parents': {}, 'thread_id': '11', 'checkpoint_ns': '', 'checkpoint_id': '1f03c345-12c9-631d-8002-04d7f52d01f1'}, created_at='2025-05-29T02:28:02.804390+00:00', parent_config={'

In [18]:
for event in graph.stream(None, fork_config, stream_mode="values"):
    event['messages'][-1].pretty_print()

================================ Human Message =================================

Human forked
================================== Ai Message ==================================

Hello from Node 3! Forked: True
================================== Ai Message ==================================

Hello from Node 4! Forked: True


In [19]:
all_states = [state for state in graph.get_state_history(thread) ]
all_states[0].values["messages"]

[HumanMessage(content='Human input to start the graph.', additional_kwargs={}, response_metadata={}, id='e4a67f5c-fb68-4653-ab6b-021d66a342bc'),
 AIMessage(content='Hello from Node 1! Forked: False', additional_kwargs={}, response_metadata={}, id='b9b8638a-1f31-429b-94e6-89fce9f56ae5'),
 HumanMessage(content='Human forked', additional_kwargs={}, response_metadata={}, id='722e38a9-d19d-48b7-81cc-2d009d4f8d6d'),
 AIMessage(content='Hello from Node 3! Forked: True', additional_kwargs={}, response_metadata={}, id='cb5f5961-5b5f-4ff7-a69e-3105dca3eb15'),
 AIMessage(content='Hello from Node 4! Forked: True', additional_kwargs={}, response_metadata={}, id='1919576a-56fd-497e-b147-0182db875835')]

In [20]:
len(all_states)

11